<a href="https://colab.research.google.com/github/michel-wue/Seminar/blob/main/plays_data_extraction1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests as r
import pandas as pd
from pandas import json_normalize
import re
from google.colab import drive
import datetime
from os import times

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [36]:
Data_basic = pd.DataFrame()
Data_basic = pd.DataFrame(columns=[
    'gameID','season','types',
    'week','HomeTeam_ID','AwayTeam_ID',
    'Winner'
    ])
week = list(range(1, 19))
seasons=list(range(2015,2023))

In [37]:


def infos(season,types,weeks):
  url = f"https://sports.core.api.espn.com/v2/sports/football/leagues/nfl/seasons/{season}/types/{types}/weeks/{weeks}/events"
  response = r.get(url)
  json_data = response.json()
  df = json_normalize(json_data['items'])
  for game in range(len(df)):
    rows=[]
    response = r.get(df['$ref'][game])
    data=response.json()
    try:
      df2=json_normalize(data['competitions'])
      rows.append(df2['id'][0])
      rows.append(season)
      rows.append(types)
      rows.append(weeks)
      rows.append(df2['competitors'][0][0]['id'])
      rows.append(df2['competitors'][0][1]['id'])
      try:
        if df2['competitors'][0][0]['winner'] == True:
          rows.append('Home')
        else:
          rows.append('Away')
      except KeyError:
        rows.append('NaN')
    except KeyError:
      print('it doesnt worked')
    Data_basic.loc[len(Data_basic)] = rows




In [38]:
for x in seasons:
  for i in week:
    infos(x,2,i)

In [40]:
xl=Data_basic.head(5)

In [41]:
xl

,GameID,season,types,week,HomeTeam_ID,AwayTeam_ID,Winner
0,400791485,2015,2,1,17,23,Home
1,400791541,2015,2,1,2,11,Home
2,400791489,2015,2,1,3,9,Away
3,400791550,2015,2,1,14,26,Home
4,400791497,2015,2,1,20,5,Home


In [42]:
playid_df = pd.DataFrame()
playid_df = pd.DataFrame(columns=['gameID', 'playID', 'awayScore', 'homeScore', 'scoreValue',
                         'statYardage', 'quarter', 'clock', 'type_text', 'team_play'
                         , 'start_down', 'start_distance', 'start_yardlinie'
                         ,'start_yardsToEndzone', 'end_down', 'end_distance', 'end_yardlinie'
                         ,'end_yardsToEndzone'])


In [33]:
def playid(GameID):
    url = f'http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/events/{GameID}/competitions/{GameID}/drives?lang=en&region=us'
    response_drive = r.get(url)
    drive_data = response_drive.json()
    drive_df = json_normalize(drive_data['items'])
    for index, row in drive_df.iterrows():
      for detail in range(len(row['plays.items'])):
        gameID = GameID
        playID = row['plays.items'][detail]['id']
        awayScore = row['plays.items'][detail]['awayScore']
        homeScore = row['plays.items'][detail]['homeScore']
        scoreValue = row['plays.items'][detail]['scoreValue']
        statYardage = row['plays.items'][detail]['statYardage']
        clock = datetime.timedelta(seconds=row['plays.items'][detail]['clock']['value'])
        quarter = row['plays.items'][detail]['period']['number']
        type_text = row['plays.items'][detail]['type']['text']


        try:
          team_play = re.search(r"teams/(\d+)", row['plays.items'][detail]['start']['team']['$ref']).group(1)
        #except TypeError:
          #print('NoTeam')
        except KeyError:
          team_play = 'NoTeam'
        start_down = row['plays.items'][detail]['start']['down']
        start_distance = row['plays.items'][detail]['start']['distance']
        start_yardlinie = row['plays.items'][detail]['start']['yardLine']
        start_yardsToEndzone = row['plays.items'][detail]['start']['yardsToEndzone']
        end_down = row['plays.items'][detail]['end']['down']
        end_distance = row['plays.items'][detail]['end']['distance']
        end_yardlinie = row['plays.items'][detail]['end']['yardLine']
        end_yardsToEndzone = row['plays.items'][detail]['end']['yardsToEndzone']
        data = [gameID,playID,awayScore,homeScore,scoreValue,statYardage, quarter, clock, type_text,
                team_play,start_down, start_distance, start_yardlinie, start_yardsToEndzone,
                end_down, end_distance,end_yardlinie,end_yardsToEndzone]
        playid_df.loc[len(playid_df)] = data

      #print(row)

In [44]:
for game in xl['GameID']:
  playid(game)

<function playid at 0x7f4add48b9a0>
Kickoff
<function playid at 0x7f4add48b9a0>
Rush
<function playid at 0x7f4add48b9a0>
Pass Reception
<function playid at 0x7f4add48b9a0>
Rush
<function playid at 0x7f4add48b9a0>
Pass Reception
<function playid at 0x7f4add48b9a0>
Rush
<function playid at 0x7f4add48b9a0>
Sack
<function playid at 0x7f4add48b9a0>
Penalty
<function playid at 0x7f4add48b9a0>
Rush
<function playid at 0x7f4add48b9a0>
Pass Reception
<function playid at 0x7f4add48b9a0>
Field Goal Missed
<function playid at 0x7f4add48b9a0>
Pass Reception
<function playid at 0x7f4add48b9a0>
Pass Reception
<function playid at 0x7f4add48b9a0>
Pass Reception
<function playid at 0x7f4add48b9a0>
Pass Incompletion
<function playid at 0x7f4add48b9a0>
Pass Incompletion
<function playid at 0x7f4add48b9a0>
Sack
<function playid at 0x7f4add48b9a0>
Punt
<function playid at 0x7f4add48b9a0>
Rush
<function playid at 0x7f4add48b9a0>
Rush
<function playid at 0x7f4add48b9a0>
Pass Reception
<function playid at 0x7f

In [34]:
playid('401326635')

<function playid at 0x7f4add48b9a0>
Kickoff
<function playid at 0x7f4add48b9a0>
Rush
<function playid at 0x7f4add48b9a0>
Pass Reception
<function playid at 0x7f4add48b9a0>
Pass Incompletion
<function playid at 0x7f4add48b9a0>
Punt
<function playid at 0x7f4add48b9a0>
Rush
<function playid at 0x7f4add48b9a0>
Pass Reception
<function playid at 0x7f4add48b9a0>
Pass Reception
<function playid at 0x7f4add48b9a0>
Rush
<function playid at 0x7f4add48b9a0>
Rush
<function playid at 0x7f4add48b9a0>
Timeout
<function playid at 0x7f4add48b9a0>
Rush
<function playid at 0x7f4add48b9a0>
Pass Reception
<function playid at 0x7f4add48b9a0>
Rush
<function playid at 0x7f4add48b9a0>
Rush
<function playid at 0x7f4add48b9a0>
Pass Reception
<function playid at 0x7f4add48b9a0>
Passing Touchdown
<function playid at 0x7f4add48b9a0>
Kickoff
<function playid at 0x7f4add48b9a0>
Rush
<function playid at 0x7f4add48b9a0>
Pass Reception
<function playid at 0x7f4add48b9a0>
Rush
<function playid at 0x7f4add48b9a0>
Rush
<fu

In [35]:
playid_df

,gameID,playID,awayScore,homeScore,scoreValue,statYardage,quarter,clock,type_text,team_play,start_down,start_distance,start_yardlinie,start_yardsToEndzone,end_down,end_distance,end_yardlinie,end_yardsToEndzone
0,401326635,40132663540,0,0,0,0,1,0 days 00:15:00,Kickoff,12,0,0,35,65,1,10,38,75
1,401326635,40132663555,0,0,0,3,1,0 days 00:15:00,Rush,4,1,10,75,75,1,10,72,72
2,401326635,40132663576,0,0,0,2,1,0 days 00:14:26,Pass Reception,4,2,7,72,72,4,1,70,70
3,401326635,401326635100,0,0,0,0,1,0 days 00:13:41,Pass Incompletion,4,3,5,70,75,1,10,70,70
4,401326635,401326635122,0,0,0,0,1,0 days 00:13:38,Punt,4,4,5,70,70,1,10,16,84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174,401326635,4013266354135,24,24,0,0,5,0 days 00:11:26,Timeout,4,1,10,12,12,1,10,12,12
175,401326635,4013266354163,24,24,0,2,5,0 days 00:10:52,Rush,4,1,10,12,12,2,8,10,10
176,401326635,4013266354184,24,24,0,-3,5,0 days 00:10:07,Rush,4,2,8,10,10,3,11,13,13
177,401326635,4013266354216,27,24,3,31,5,0 days 00:09:22,Field Goal Good,4,3,11,13,13,-1,10,0,65
